In [1]:
from scipy.io import loadmat
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split ,GridSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.io import loadmat
import numpy as np

# Load the .mat file
data = loadmat(r'C:\Users\vishw\Documents\karafeature\MM08.mat')
lab =loadmat(r'C:\Users\vishw\Documents\karafeature\LabelMM081.mat')
# Extract features and labels
features = data['all_features']
labels = lab['all_labels'].flatten()
# labels = lab['all_labels'].flatten()  # Flatten if labels are in (8184, 1) shape
# If labels start from 1, adjust them to start from 0
if np.min(labels) == 1:
    labels -= 1

# scaler = StandardScaler()
# features = scaler.fit_transform(features)
# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [4]:
import csv
import time
import pandas as pd
import numpy as np
import json
import csv
# import sklearn.cluster.k_means_ as KMeans
from sklearn.cluster import KMeans
import warnings
from collections import Counter
# from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

warnings.filterwarnings("ignore")  # ignore warning


class GranularBall:
    """class of the granular ball"""
    def __init__(self, data):
        """
        :param data:  Labeled data set, the "-2" column is the class label, the last column is the index of each line
        and each of the preceding columns corresponds to a feature
        """
        self.data = data[:, :]
        self.data_no_label = data[:, :-2]
        self.num, self.dim = self.data_no_label.shape
        self.center = self.data_no_label.mean(0)
        self.label, self.purity ,self.r= self.__get_label_and_purity_and_r()

    def __get_label_and_purity_and_r(self):
        """
        :return: the label and purity of the granular ball.
        """
        count = Counter(self.data[:, -2])
        label = max(count, key=count.get)
        purity = count[label] / self.num
        arr=np.array(self.data_no_label)-self.center
        ar=np.square(arr)
        a=np.sqrt(np.sum(ar,1))
        r=np.sum(a)/len(self.data_no_label)
        return label, purity,r

    def split_2balls(self):
        """
        split the granular ball to 2 new balls by using 2_means.
        """
        # label_cluster = KMeans(X=self.data_no_label, n_clusters=2)[1]
        clu=KMeans(n_clusters=2).fit(self.data_no_label)
        label_cluster=clu.labels_
        if sum(label_cluster == 0) and sum(label_cluster == 1):
            ball1 = GranularBall(self.data[label_cluster == 0, :])
            ball2 = GranularBall(self.data[label_cluster == 1, :])
        else:
            ball1 = GranularBall(self.data[0:1, :]) 
            ball2 = GranularBall(self.data[1:, :])
        return ball1, ball2


class GBList:
    """class of the list of granular ball"""
    def __init__(self, data=None):
        self.data = data[:, :]
        self.granular_balls = [GranularBall(self.data)]  # gbs is initialized with all data

    def init_granular_balls(self, purity=1, min_sample=1):
        """
        Split the balls, initialize the balls list.
        :param purity: If the purity of a ball is greater than this value, stop splitting.
        :param min_sample: If the number of samples of a ball is less than this value, stop splitting.
        """
        ll = len(self.granular_balls)
        i = 0
        while True:
            if self.granular_balls[i].purity < purity and self.granular_balls[i].num > min_sample:
                split_balls = self.granular_balls[i].split_2balls()
                self.granular_balls[i] = split_balls[0]
                self.granular_balls.append(split_balls[1])
                ll += 1
            else:
                i += 1
            if i >= ll:
                break
        self.data = self.get_data()

    def get_data_size(self):
        return list(map(lambda x: len(x.data), self.granular_balls))

    def get_purity(self):
        return list(map(lambda x: x.purity, self.granular_balls))

    def get_center(self):
        """
        :return: the center of each ball.
        """
        return np.array(list(map(lambda x: x.center, self.granular_balls)))

    def get_r(self):
        """
        :return: 返回半径r
        """
        return np.array(list(map(lambda x: x.r, self.granular_balls)))

    def get_data(self):
        """
        :return: Data from all existing granular balls in the GBlist.
        """
        list_data = [ball.data for ball in self.granular_balls]
        return np.vstack(list_data)
    def del_ball(self,purty=0.,num_data=0):
        T_ball=[]
        for ball in self.granular_balls:
            if ball.purity>=purty and ball.num>=num_data:
                T_ball.append(ball)
        self.granular_balls=T_ball.copy()
        self.data=self.get_data()
    def re_division(self, i):
        """
        Data division with the center of the ball.
        :return: a list of new granular balls after divisions.
        """
        k = len(self.granular_balls)
        attributes = list(range(self.data.shape[1] - 2))
        attributes.remove(i)
        clu = KMeans(n_clusters=k, init=self.get_center()[:, attributes], max_iter=1).fit(self.data[:, attributes])
        label_cluster = clu.labels_
        # label_cluster = KMeans(X=self.data[:, attributes], n_clusters=k,
        #                         init=self.get_center()[:, attributes], max_iter=1)[1]
        granular_balls_division = []
        for i in set(label_cluster):
            granular_balls_division.append(GranularBall(self.data[label_cluster == i, :]))
        return granular_balls_division
def generate_ball_data(data,pur,delbals):
    num, dim = data[:, :-1].shape
    index = np.array(range(num)).reshape(num, 1)  # column of index
    data = np.hstack((data, index))  # Add the index column to the last column of the data
    # step 1.
    #print(data[0:4])
    gb = GBList(data)  # create the list of granular balls
    gb.init_granular_balls(purity=pur)  # initialize the list
    gb.del_ball(num_data=delbals)
    centers=gb.get_center().tolist()
    rs=gb.get_r().tolist()
    # print(type(centers[0]))
    balldata = []  # 检验
    for i in range(len(gb.granular_balls)):
        a=[]
        a.append(centers[i])
        a.append(rs[i])
        # print(data[i][-2])
        if gb.granular_balls[i].label==-1:
            a.append(-1)
        elif gb.granular_balls[i].label==1:
            a.append(1)
        balldata.append(a)
    # print(balldata[0])
    return balldata
def gen_balls(data,pur,delbals):
    # df=pd.read_csv(url,header=None)
    # data=df.values
    # print(data.shape)
    balls=generate_ball_data(data,pur=pur,delbals=delbals)
    R_balls=[]
    for i in balls:
        t_ball=[]
        t_ball.append(i[0])
        t_ball.append(i[1])
        t_ball.append(i[2])
        R_balls.append(t_ball)
    return R_balls


In [5]:
import numpy as np
import time
from sklearn.preprocessing import StandardScaler

# Optimized activation functions using NumPy
def selu(x):
    alpha = 1.6732632423543772
    scale = 1.0507009873554805
    return scale * np.where(x > 0, x, alpha * (np.exp(x) - 1))

def leaky_relu(x):
    return np.where(x > 0, x, 0.1 * x)

def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def hardlim(x):
    return np.where(x > 0, 1, 0)

def tribas(x):
    return np.maximum(1 - np.abs(x), 0)

def radbas(x):
    return np.exp(-x**2)

# Mapping activation functions to their respective functions
activation_functions = {
    1: selu,
    2: relu,
    3: sigmoid,
    4: np.sin,
    5: hardlim,
    6: tribas,
    7: radbas,
    8: np.sign,
    9: leaky_relu,
    10: np.tanh
}

# One-hot encoding function
def one_hot(x, n_class):
    y = np.zeros((len(x), n_class))
    for i in range(n_class):
        y[x == i, i] = 1
    return y

# Evaluation metrics
def Evaluate(ACTUAL, PREDICTED):
    p = np.sum(ACTUAL == 1)
    n = np.sum(ACTUAL == 0)
    
    tp = np.sum((ACTUAL == 1) & (PREDICTED == 1))
    tn = np.sum((ACTUAL == 0) & (PREDICTED == 0))
    fp = np.sum((ACTUAL == 0) & (PREDICTED == 1))
    fn = np.sum((ACTUAL == 1) & (PREDICTED == 0))
    
    accuracy = 100 * (tp + tn) / (p + n)
    sensitivity = 100 * tp / p if p != 0 else 0
    specificity = 100 * tn / n if n != 0 else 0
    precision = 100 * tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = sensitivity
    f_measure = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0
    gmean = 100 * np.sqrt((tp / p) * (tn / n)) if p != 0 and n != 0 else 0
    
    return [accuracy, sensitivity, specificity, precision, recall, f_measure, gmean, tp, tn, fp, fn]

def RVFL_train(train_data, test_data, C, N, activation):
    start = time.time()
    
    # Split training data
    trainX = train_data[:, :-1]
    trainY = train_data[:, -1]
    
    # One-hot encode labels
    nclass = 2
    trainY_encoded = one_hot(trainY.astype(int), nclass)
    
    # Initialize random weights and biases
    Nsample, Nfea = trainX.shape
    W = np.random.uniform(-1, 1, (Nfea, N))
    b = 0.1 * np.random.rand(1, N)
    
    # Compute hidden layer output for training data
    X1 = np.dot(trainX, W) + b
    activation_func = activation_functions.get(activation, relu)
    X1 = activation_func(X1)
    
    # Augment the input with hidden layer output and bias term
    X = np.hstack((trainX, X1, np.ones((Nsample, 1))))
    
    # Compute output weights (beta) using regularized least squares
    if X.shape[1] < Nsample:
        beta = np.linalg.inv(np.eye(X.shape[1]) / C + X.T @ X) @ X.T @ trainY_encoded
    else:
        beta = X.T @ np.linalg.inv(np.eye(X.shape[0]) / C + X @ X.T) @ trainY_encoded

    # Test phase
    testX = test_data[:, :-1]
    testY = test_data[:, -1]
    
    X1_test = np.dot(testX, W) + b
    X1_test = activation_func(X1_test)
    
    # Augment test data
    X_test = np.hstack((testX, X1_test, np.ones((testX.shape[0], 1))))
    
    # Calculate scores and classify
    raw_scores = X_test @ beta
    predicted_labels = np.argmax(raw_scores, axis=1)
    
    # Evaluate performance
    eval_metrics = Evaluate(testY, predicted_labels)
    end = time.time()
    
    return eval_metrics, end - start


In [6]:
import os
import numpy as np
# from gen_ball import gen_balls
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

# Assuming 'features' and 'labels' are already loaded
# Ensure labels are in the right shape
# labels = labels.flatten()

# Combine features and labels into one dataset
dataset = np.hstack((features, labels.reshape(-1, 1)))

# Convert label 0 to -1 for compatibility with gen_balls
dataset[dataset[:, -1] == 0, -1] = -1

# Shuffle the dataset
np.random.seed(0)
np.random.shuffle(dataset)

# Split data into training (70%) and testing (30%) sets
train_size = int(0.7 * len(dataset))
AA_train = dataset[:train_size]
A_test = dataset[train_size:]

# Convert -1 labels back to 0 for the test set
A_test[A_test[:, -1] == -1, -1] = 0

# Generate synthetic balls for training data
pur = 1 - (0.015 * 5)
num = 2
A_train = gen_balls(AA_train, pur=pur, delbals=num)

# Extract Centers, Radii, and Labels
Centers = np.array([item[0] for item in A_train])
Labels = np.array([item[-1] for item in A_train]).reshape(-1, 1)

# Prepare the final training data
A_train = np.hstack((Centers, Labels))

# Convert -1 labels back to 0 for the training set
A_train[A_train[:, -1] == -1, -1] = 0

# Feature Scaling
scaler = StandardScaler()
A_train[:, :-1] = scaler.fit_transform(A_train[:, :-1])
A_test[:, :-1] = scaler.transform(A_test[:, :-1])

# RVFL parameters (these can be tuned further)
c1_best = 0.01
N_best = 203
Act_best = 8

# Implementing K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=0)
accuracy_scores = []

for train_index, val_index in kf.split(A_train):
    fold_train, fold_val = A_train[train_index], A_train[val_index]
    
    # Train the RVFL model on the fold
    Eval, Test_time = RVFL_train(fold_train, A_test, c1_best, N_best, Act_best)
    
    # Capture the test accuracy
    Test_accuracy = Eval[0]
    accuracy_scores.append(Test_accuracy)
    print(f"Fold Test Accuracy: {Test_accuracy:.2f}%")

# Average accuracy across all folds
avg_accuracy = np.mean(accuracy_scores)
print(f"\nAverage Test Accuracy: {avg_accuracy:.2f}%")


IndexError: list index out of range